In [38]:
import re
import glob
import json
import string
import pymorphy2
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

path = "/data/share/lab05data"

### Импорт и обработка тестовой выборки

In [2]:
text_id = [1, 3587, 770, 3593, 3084, 3607, 536, 1561, 2588, 3931, 2079, 2481, 2406, 1580, 2605, 2101, 55, 1081, 62, 63, 3649, 3139, 581, 70, 3143, 1098, 1099, 589, 2134, 88, 2649, 357, 2139, 1117, 358, 1634, 189, 1644, 109, 3184, 3188, 2166, 2682, 3198, 3125, 3203, 1669, 646, 3719, 1560, 144, 3730, 1174, 3223, 3738, 3739, 1183, 3744, 1188, 1701, 711, 680, 535, 2174, 1714, 799, 695, 1720, 2462, 1211, 701, 702, 1215, 630, 194, 3783, 3784, 3279, 1150, 2259, 724, 383, 3800, 2265, 3293, 3296, 740, 1767, 3820, 3822, 3824, 241, 3317, 1270, 2006, 1281, 231, 812, 2311, 2830, 2323, 789, 2010, 409, 133, 1311, 2343, 3368, 2860, 3886, 2864, 307, 2357, 2358, 2954, 2362, 3901, 1343, 822, 1345, 1347, 324, 3639, 2377, 312, 3405, 2896, 3300, 3418, 859, 861, 352, 3426, 356, 3429, 2918, 658, 2924, 1909, 2423, 1344, 1407, 2436, 1928, 1418, 3566, 3481, 1428, 406, 1432, 2457, 2288, 2461, 1691, 2977, 2467, 1956, 2981, 2982, 2984, 1961, 939, 429, 2992, 433, 3067, 2036, 2485, 444, 1469, 3920, 3524, 455, 504, 2028, 3025, 3538, 2517, 3542, 2521, 869, 3547, 336, 3038, 2531, 998, 2535, 2025, 1704, 2029, 2030, 510, 1521, 2548, 1525, 870, 836, 1024, 2731, 2646]

rows_list_test = []
for i in text_id:
    # test
    text_file = open(path + "/test_" + str(i) + ".txt", "r")
    if text_file.mode == 'r':
        contents =text_file.read()
    row = {"id":i, "text":contents}
    rows_list_test.append(row)

df_text = pd.DataFrame(rows_list_test, columns=["id","text"])
df_text.head()

,id,text
0,1,<p><strong>В наши магазины приходят за красивы...
1,3587,<strong>Обязанности:</strong> <ul> <li>Выкладк...
2,770,<p><strong>телефон для собеседования +7 978 00...
3,3593,<p><strong>Специалист по товародвижению в ночн...
4,3084,Обязанности: продажа натуральной лечебной прод...


In [3]:
txt_files = glob.glob(path + "/base_*.txt")
rows_list_base = []
for file in txt_files:    
    open_file = open(file, "r")
    if open_file.mode == 'r':
        contents =open_file.read()
    row = {"text":contents}
    rows_list_base.append(row)

df_text_base = pd.DataFrame(rows_list_base, columns=["text"])
df_text_base.head()

,text
0,Крупная инвестиционная компания ищет в свою ко...
1,"<p>NetCracker Technology Corp., a large softwa..."
2,<p>ИТ-компания ищет в свою команду талантливог...
3,<p>Ищем прекрасного программиста 1С в дружный ...
4,<p>Предлагаем уникальную возможность присоедин...


In [4]:
def remove_html_tags(text):
    """Удалим html теги из текста"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_special_chars(text):
    """Удалим все кроме букв и пробелов"""
    clean = re.sub('[^А-Яа-я[" "]]+', '', text)
    clean = clean.translate(str.maketrans('', '', string.punctuation))
    return clean

def remove_digit_from_text(text):
    """Удалим цифры из текста"""
    clean = re.sub(r'\d+', '', text)
    return clean

def transform_to_lower(text):
    """Приведем весь текст к нижнему регистру"""
    return text.lower()

In [5]:
df_text["text"] = df_text["text"].apply(lambda x: remove_html_tags(x))
df_text["text"] = df_text["text"].apply(lambda x: remove_special_chars(x))
df_text["text"] = df_text["text"].apply(lambda x: transform_to_lower(x))
df_text["text"] = df_text["text"].apply(lambda x: remove_digit_from_text(x))

df_text_base["text"] = df_text_base["text"].apply(lambda x: remove_html_tags(x))
df_text_base["text"] = df_text_base["text"].apply(lambda x: remove_special_chars(x))
df_text_base["text"] = df_text_base["text"].apply(lambda x: transform_to_lower(x))
df_text_base["text"] = df_text_base["text"].apply(lambda x: remove_digit_from_text(x))

In [6]:
# Токенизируем
df_text["text"] = df_text["text"].apply(lambda x: word_tokenize(x))
df_text_base["text"] = df_text_base["text"].apply(lambda x: word_tokenize(x))
df_text.head()

,id,text
0,1,"[в, наши, магазины, приходят, за, красивыми, и..."
1,3587,"[обязанности, выкладка, товаров, на, витрины, ..."
2,770,"[телефон, для, собеседования, евгений, руковод..."
3,3593,"[специалист, по, товародвижению, в, ночное, вр..."
4,3084,"[обязанности, продажа, натуральной, лечебной, ..."


In [8]:
# Удаляем стоп-слова
df_text["text"] = df_text["text"].apply(lambda word_list: [word for word in word_list if word not in stopwords.words('russian')])
df_text_base["text"] = df_text_base["text"].apply(lambda word_list: [word for word in word_list if word not in stopwords.words('russian')])
df_text.head()

,id,text
0,1,"[наши, магазины, приходят, красивыми, удобными..."
1,3587,"[обязанности, выкладка, товаров, витрины, пров..."
2,770,"[телефон, собеседования, евгений, руководитель..."
3,3593,"[специалист, товародвижению, ночное, время, тц..."
4,3084,"[обязанности, продажа, натуральной, лечебной, ..."


In [9]:
#проведем лемматизацию
pymorph = pymorphy2.MorphAnalyzer()

df_text["text"] = df_text["text"].apply(lambda word_list: [pymorph.parse(word)[0].normal_form for word in word_list])
df_text_base["text"] = df_text_base["text"].apply(lambda word_list: [pymorph.parse(word)[0].normal_form for word in word_list])
df_text.head()

,id,text
0,1,"[наш, магазин, приходить, красивый, удобный, п..."
1,3587,"[обязанность, выкладка, товар, витрина, провер..."
2,770,"[телефон, собеседование, евгений, руководитель..."
3,3593,"[специалист, товародвижение, ночное, время, тц..."
4,3084,"[обязанность, продажа, натуральный, лечебный, ..."


In [16]:
# Обучаем модель
vect = TfidfVectorizer(min_df=1)  
def cosine_sim(text1, text2):
    tfidf = vect.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [30]:
corpus = []
for index, row in df_text_base.iterrows():
    for word in row["text"]:
        corpus.append(word)

In [70]:
output = []
for index, row in df_text.iterrows():
    row = {"id":row["id"], "cos":cosine_sim(' '.join(word for word in row["text"]), ' '.join(word for word in corpus))}
    output.append(row)

answer = pd.DataFrame(output)
print(answer.info())
answer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
cos    200 non-null float64
id     200 non-null int64
dtypes: float64(1), int64(1)
memory usage: 3.2 KB
None


,cos,id
0,0.100822,1
1,0.135880,3587
2,0.069648,770
3,0.134977,3593
4,0.101592,3084


In [74]:
defined = []
other = []
cos_avg = answer["cos"].mean()

for index, row in answer.iterrows():
    if row["cos"] >= cos_avg:
        defined.append(int(row["id"]))
    else:
        other.append(int(row["id"]))

json_dict = {"defined":defined,"other":other}

In [75]:
with open('../lab05.json', 'w') as f:
    json.dump(json_dict, f, sort_keys=True)

In [76]:
cat ../lab05.json

{"defined": [3607, 536, 55, 3649, 3139, 1098, 1099, 589, 2134, 88, 357, 2139, 358, 1644, 3188, 2166, 3198, 3203, 646, 3719, 3730, 3223, 3738, 3739, 1188, 680, 535, 1211, 701, 1215, 194, 3784, 2259, 3293, 740, 1767, 3820, 3822, 3824, 241, 3317, 1270, 2006, 1281, 2311, 789, 1311, 2343, 3368, 3886, 2864, 2358, 2954, 2362, 3901, 822, 1347, 324, 3405, 2896, 3418, 861, 3426, 3429, 2918, 2924, 2423, 2436, 1928, 1418, 3566, 1428, 1432, 2288, 2977, 939, 429, 2992, 3524, 455, 2517, 3542, 2521, 869, 336, 3038, 2531, 2535, 2025, 1704, 2030, 510, 2548, 2731, 2646], "other": [1, 3587, 770, 3593, 3084, 1561, 2588, 3931, 2079, 2481, 2406, 1580, 2605, 2101, 1081, 62, 63, 581, 70, 3143, 2649, 1117, 1634, 189, 109, 3184, 2682, 3125, 1669, 1560, 144, 1174, 1183, 3744, 1701, 711, 2174, 1714, 799, 695, 1720, 2462, 702, 630, 3783, 3279, 1150, 724, 383, 3800, 2265, 3296, 231, 812, 2830, 2323, 2010, 409, 133, 2860, 307, 2357, 1343, 1345, 3639, 2377, 312, 3300, 859, 352, 356, 658, 1909, 1344, 1407, 3481, 406, 2